# Experiment with extracting shape info from confocal images of fish embryos

In [ ]:
from aicsimageio import AICSImage
import os 
import glob2 as glob
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)
import plotly.express as px
from skimage import data
from skimage import filters
from skimage.segmentation import active_contour
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from skimage.morphology import black_tophat, skeletonize, convex_hull_image 
from skimage.morphology import disk
import skimage
from matplotlib.pyplot import colorbar 
from matplotlib.pyplot import savefig

save_path_depth = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/morphSeq/data/20230412/3D_depth/"
# make save directory
if not os.path.isdir(save_path_depth):
    os.makedirs(save_path_depth)
    

image_path = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/morphSeq/data/20230412/timeseries.nd2"
imObject = AICSImage(image_path)

imObject.set_scene("XYPos:17")
n_time_points = imObject.dims["T"][0]
res_raw = imObject.physical_pixel_sizes
res_array = np.asarray(res_raw)

# generate and save depth image
n_time_points = imObject.dims["T"][0]

# im_array = np.empty((n_time_points, 317*818))
for t in range(n_time_points):

    # extract image
    imData = np.squeeze(imObject.get_image_data("CZYX", T=t))
    imData_rs = scipy.ndimage.zoom(imData, [1, 1/4, 1/4])
    
    z_grid3, _, _ = np.meshgrid(range(0, imData_rs.shape[0]), 
                                        range(0, imData_rs.shape[1]), 
                                        range(0, imData_rs.shape[2]),
                                       indexing="ij")
    
    max_z_plane = np.argmax(imData_rs, axis=0)
    max_z_b = np.max(imData_rs, axis=0)
    
    threshold_sa = filters.threshold_sauvola(max_z_b, window_size=11)

    # this should result in a sparse cloud of white regions that jointly define the embryo body
    fish_mask = (max_z_b < threshold_sa)*1

    # use morphological closure operation to fill in shape
    footprint = disk(18)
    fish_closed = closing(fish_mask, footprint)

    # remove small objects 
    fish_strip = skimage.morphology.remove_small_objects(label(fish_closed), min_size=64)

    # erode to
    fish_clean = skimage.morphology.binary_erosion(fish_strip, footprint)
    
    
    max_z_plane = max_z_plane*20
    mm = np.max(max_z_plane)
    max_z_plane[np.where(fish_clean==0)] = mm
    
    fig = plt.figure()
    
    im_plot = mm-max_z_plane

    ax = plt.imshow(np.flipud(im_plot))
    
    cbar = colorbar(label='height of surface (microns)', shrink=0.5, boundaries= [0, 300])
    
    fig.savefig(save_path_depth + f'depth_T{t:03}.tif')
#     fig.show()
    
#     im_array[t, :] = max_z_plane.flatten()

In [ ]:
# im_array = np.empty((n_time_points, 317*818))
for t in range(n_time_points):

    # extract image
    imData = np.squeeze(imObject.get_image_data("CZYX", T=t))
    imData_rs = scipy.ndimage.zoom(imData, [1, 1/4, 1/4])
    
    z_grid3, _, _ = np.meshgrid(range(0, imData_rs.shape[0]), 
                                        range(0, imData_rs.shape[1]), 
                                        range(0, imData_rs.shape[2]),
                                       indexing="ij")
    
    max_z_plane = np.argmax(imData_rs, axis=0)
    max_z_b = np.max(imData_rs, axis=0)
    
    threshold_sa = filters.threshold_sauvola(max_z_b, window_size=11)

    # this should result in a sparse cloud of white regions that jointly define the embryo body
    fish_mask = (max_z_b < threshold_sa)*1

    # use morphological closure operation to fill in shape
    footprint = disk(15)
    fish_closed = closing(fish_mask, footprint)

    # remove small objects 
    fish_strip = skimage.morphology.remove_small_objects(label(fish_closed), min_size=64)

    # erode to
    fish_clean = skimage.morphology.binary_erosion(fish_strip, footprint)
    
    
    max_z_plane = max_z_plane*20
    mm = np.max(max_z_plane)
    max_z_plane[np.where(fish_clean==0)] = mm
    
    fig = plt.figure()
    
    im_plot = mm-max_z_plane

    ax = plt.imshow(np.flipud(im_plot))
    
    cbar = colorbar(label='height of surface (microns)', shrink=0.5)
    plt.clim(0, 300)
    
    fig.savefig(save_path_depth + f'depth_T{t:03}.tif')

In [ ]:
res_array

In [ ]:
import umap
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
# apply UMAP dimensionality reduction (why not?)

reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(im_array)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(embedding[:, 0], embedding[:, 1], color=range(n_time_points))
fig.show()

In [ ]:
from matplotlib import pyplot as plt

z_grid3, y_grid3, x_grid3 = np.meshgrid(range(0, imData.shape[0]), 
                                        range(0, imData.shape[1]), 
                                        range(0, imData.shape[2]),
                                        indexing="ij")

mean_z_plane = np.divide(np.sum(np.multiply(imData, z_grid3), axis=0), np.sum(imData, axis=0))
maxmean_z_plane = mean_z_plane.astype(float)

max_z_plane = np.argmax(imData, axis=0)

plt.imshow(max_z_plane)

In [ ]:
import umap
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
# apply UMAP dimensionality reduction (why not?)

reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(max_z_plane.flatten())

Attempt to infer a surface using marching cubes

**Can we use linear interpolation to fill label gaps?**

In [ ]:
from skimage.morphology import disk
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)

# first, generate 3D mask to identify interior pixels that we wish to interpolate
fish_mask = (np.max(labelDataBin, axis=0) == 1)*1

# use morphological closure operation to fill in shape
footprint = disk(8)
fp_small = disk(2)
fish_closed = closing(fish_mask > 0, footprint)
fish_clean = skimage.morphology.remove_small_objects(label(fish_closed), min_size=64)
fish_eroded = skimage.morphology.binary_erosion(fish_clean, fp_small)
# plt.imshow(fish_mask)
plt.imshow(fish_clean)

Use known z label locations to interpolate missing ones

In [ ]:
from scipy.interpolate import LinearNDInterpolator

y_grid2, x_grid2 = np.meshgrid(
                                range(0, max_z_plane.shape[0]), 
                                range(0, max_z_plane.shape[1]),
                                indexing="ij")

x_flat = x_grid2.flatten()
y_flat = y_grid2.flatten()

# generate diplacement vectors
max_z_plane_vals = max_z_plane.flatten()
mask_flat = (fish_clean.flatten() > 0)*1
mask_er = (fish_eroded.flatten() > 0)*1

ref_x = x_flat[np.where((~np.isnan(max_z_plane_vals)) & (mask_flat==1))]
ref_y = y_flat[np.where((~np.isnan(max_z_plane_vals)) & (mask_flat==1))]
ref_z = max_z_plane_vals[np.where((~np.isnan(max_z_plane_vals)) & (mask_flat==1))]

query_x = x_flat[np.where(mask_flat==1)]
query_y = y_flat[np.where(mask_flat==1)]
query_x_er = x_flat[np.where(mask_er==1)]
query_y_er = y_flat[np.where(mask_er==1)]

# interpolate
interp_z = LinearNDInterpolator(list(zip(ref_x, ref_y)), ref_z)
query_z = interp_z(query_x, query_y)
            

# disp_x = interp_x(source_xyz_norm[:, 0], source_xyz_norm[:, 1], source_xyz_norm[:, 2])
# disp_y = interp_y(source_xyz_norm[:, 0], source_xyz_norm[:, 1], source_xyz_norm[:, 2])
# disp_z = interp_z(source_xyz_norm[:, 0], source_xyz_norm[:, 1], source_xyz_norm[:, 2])

In [ ]:
surf_mask_array_raw = np.zeros(labelData_rs.shape)
surf_mask_array_raw[np.round(query_z).astype(int), query_y, query_x] = 1

surf_mask_array = surf_mask_array_raw.copy() 
# surf_mask_array[40, query_y_er, query_x_er] = 1

plt.imshow(np.argmax(surf_mask_array, axis=0))

Maybe alphashape?

In [ ]:
from alphashape import alphashape
import open3d as o3d
z_flat = max_z_plane.flatten()
plot_x = x_flat[np.where((~np.isnan(max_z_plane_vals)))]
plot_y = y_flat[np.where((~np.isnan(max_z_plane_vals)))]
plot_z = z_flat[np.where((~np.isnan(max_z_plane_vals)))]

surf_flat = surf_mask_array.flatten()
keep_indices = np.where(surf_mask_array>0)
# x3 = x_grid3[keep_indices]
# y3 = y_grid3[keep_indices]
# z3 = z_grid3[keep_indices]
# generate smooth bottom
base_array = np.empty((len(plot_x), 3))
base_array[:, 0] = plot_x
base_array[:, 1] = plot_y
base_array[:, 2] = plot_z + 10

xyz_array3 = np.concatenate((plot_x[:, np.newaxis], plot_y[:, np.newaxis], plot_z[:, np.newaxis]), axis=1)
# xyz_array3 = np.concatenate((x_surf3[:, np.newaxis], y_surf3[:, np.newaxis], z_surf3[:, np.newaxis]), axis=1)
xyz_array3 = np.concatenate((xyz_array3, base_array), axis=0)
xyz_array3_norm = xyz_array3.copy()
xyz_array3_norm = xyz_array3_norm / np.max(xyz_array3_norm)

print(xyz_array3_norm.shape)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_array3_norm)
pcd_ds = pcd.voxel_down_sample(voxel_size=0.0001)

xyz_ds = np.asarray(pcd_ds.points)
print(xyz_ds.shape)
alpha_fish = alphashape(xyz_ds, alpha=50)
alpha_fish.show()

Well...let's try a different method

In [ ]:
from skimage.filters import gaussian

surf_mask_array_smooth = np.argmax(surf_mask_array_raw.copy(), axis=0)
surf_mask_array_smooth[np.where(surf_mask_array_smooth==0)] = 84
surf_mask_array_smooth = gaussian(surf_mask_array_smooth, sigma=2, preserve_range=True)
plt.imshow(surf_mask_array_smooth)
new_z = surf_mask_array_smooth[query_y_er, query_x_er]

In [ ]:
xyz_array = np.concatenate((query_x_er[:, np.newaxis], query_y_er[:, np.newaxis], new_z[:, np.newaxis]), axis=1)

base_array = np.empty((len(query_x_er), 3))
base_array[:, 0] = query_x_er
base_array[:, 1] = query_y_er
base_array[:, 2] = new_z + 20

# xyz_array3 = np.concatenate((plot_x[:, np.newaxis], plot_y[:, np.newaxis], plot_z[:, np.newaxis]), axis=1)
# xyz_array3 = np.concatenate((x_surf3[:, np.newaxis], y_surf3[:, np.newaxis], z_surf3[:, np.newaxis]), axis=1)
xyz_array3 = np.concatenate((xyz_array, base_array), axis=0)
xyz_array3_norm = xyz_array3.copy()
xyz_array3_norm = xyz_array3_norm / np.max(xyz_array3_norm)

print(xyz_array3_norm.shape)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_array3_norm)
pcd_ds = pcd.voxel_down_sample(voxel_size=0.0001)
cl, ind = pcd_ds.remove_statistical_outlier(nb_neighbors=20,
                                                        std_ratio=2.0)

xyz_ds = np.asarray(pcd_ds.points)
print(xyz_ds.shape)
alpha_fish = alphashape(xyz_ds[ind], alpha=50)
alpha_fish.show()

In [ ]:
alpha_fish

In [ ]:
import pyvista as pv
import open3d as o3d

xyz_array = np.concatenate((query_x_er[:, np.newaxis], query_y_er[:, np.newaxis], new_z[:, np.newaxis]), axis=1)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_array)
pcd_ds = pcd.voxel_down_sample(voxel_size=1)

xyz_ds = np.asarray(pcd_ds.points)
pv_cloud = pv.PolyData(xyz_ds)
print(xyz_ds.shape)
print(xyz_array.shape)


alpha = 10
dl_3d_mesh = pv_cloud.delaunay_2d(alpha=alpha)
# # dl_3d_mesh = alphashape.alphashape(xyz_in, alpha=8)
# dl_3d_mesh.fill_holes(200, inplace=True)
smooth = dl_3d_mesh.smooth_taubin(n_iter=5000)
smooth.plot(jupyter_backend='ipygany', scalars=xyz_ds[:, 2])

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(int)

In [ ]:
(mnist.data.shape)

In [ ]:
nppas.fill_holes_in_surface(smoothed_surface, size_limit=300)

In [ ]:
simplified_surface2 = nppas.decimate_quadric(smoothed_surface, fraction=0.01)
simplified_surface2

**Let's see if we can fill gaps using linear interpolation**

In [ ]:
from skimage.morphology import disk
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)

# first, generate 3D mask to identify interior pixels that we wish to interpolate
fish_mask = (np.max(imDataBin, axis=0) == 1)*1

# use morphological closure operation to fill in shape
footprint = disk(15)
fish_closed = closing(fish_mask, footprint)

# plt.imshow(fish_mask)
plt.imshow(fish_closed)


In [ ]:
from scipy import interpolate

# identify "holes" for which not a single pixel nonzero exists 
query_indices = np.where((fish_closed==1) & (fish_mask!=1))[0]
ref_indices = np.where(fish_mask==1)[0]
print(len(candidate_indices))
print(len(ref_indices))

f = interpolate.interp2d(xyz_array2[:, 0], xyz_array2[:, 1], xyz_array2[:, 2], kind='linear')
x_query = x_grid2[query_indices]
y_query = y_grid2[query_indices]
z_query = f(x_query, y_query)

Try fitting to the surface points only

In [ ]:
surf_mask_array = np.zeros(imData_rs.shape)
sub_i = np.roll(xyz_array2, 1, axis=1)
surf_indices = np.ravel_multi_index((1,2,2), imData_rs.shape)
surf_mask_array[xyz_array2[:, 2], xyz_array2[:, 1], xyz_array2[:, 0]] = 1

e_surf = nppas.all_labels_to_surface(surf_mask_array)
e_surf

In [ ]:
e_surf.__dict__.keys()

**Attempt 2:** downsample, interpolate, and then infer mesh

In [ ]:
# import numpy as np
# import scipy
# from scipy import interpolate


# f = interpolate.interp2d(xyz_ds_in[:, 0], xyz_ds_in[:, 1], xyz_ds_in[:, 2], kind='linear')


In [ ]:
# npoints = 150
# xg = np.linspace(np.min(xyz_ds_in[:, 0]), np.max(xyz_ds_in[:, 0]), npoints)
# yg = np.linspace(np.min(xyz_ds_in[:, 1]), np.max(xyz_ds_in[:, 1]), npoints)

# x_grid, y_grid = np.meshgrid(xg, yg)

# znew = f(xg, yg)

# xyz_interp = np.concatenate((np.reshape(x_grid, (x_grid.size, 1)), 
#                              np.reshape(y_grid, (x_grid.size, 1)),
#                              np.reshape(znew, (x_grid.size, 1))),
#                              axis=1)

# xyz_interp_filt = xyz_interp[np.where(xyz_interp[:, 2]<=350)]
# xyz_interp_filt = xyz_interp_filt[xyz_interp_filt[:, 2]>0]
# pv_cloud_interp = pv.PolyData(xyz_interp_filt)
# pv_cloud_interp.plot(jupyter_backend='ipygany')